# Introduction to Spark

In this notebook, you will be introduced to the Apache Spark libary for big data processing. There's a now a python package called `pyspark` which will load Spark for you. Tthe variable `sc` is a Spark context that lets you interact with the Spark runtime. Check that it is correctly initialised:

In [69]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
print(sc)
print("Ready to go!")

<SparkContext master=local[*] appName=pyspark-shell>
Ready to go!


### Learning activity: Create RDDs

To analyse large datasets using Spark you will load them into Resilient Distributed Datasets (RDDs). There are a number of ways in which you can create RDDs. Use the `parallelize()` function to create one from a Python collection of elements ["Hello", "World", "!"] 

In [70]:
list = ["Hello", "World", "!"]
rdd = sc.parallelize(list)

Use the `textFile()` function to create an RDD from the file `war-and-peace.txt` and store it in a variable called `lines`. 

In [71]:
lines = sc.textFile("war-and-peace.txt")

### Learning activity: Basic RDD manipulation

Print the number of lines in War and Peace using the functions `count()`.

In [72]:
print(lines.count())

54223


Print the first 15 lines using the functions `take()`.

In [73]:
print(lines.take(15))

['                                      1869', '                                 WAR AND PEACE', '                                 by Leo Tolstoy', 'BK1', '                                 BOOK ONE: 1805', 'BK1|CH1', '  CHAPTER I', '', '  "Well, Prince, so Genoa and Lucca are now just family estates of the', "Buonapartes. But I warn you, if you don't tell me that this means war,", 'if you still try to defend the infamies and horrors perpetrated by', 'that Antichrist- I really believe he is Antichrist- I will have', 'nothing more to do with you and you are no longer my friend, no longer', "my 'faithful slave,' as you call yourself! But how do you do? I see", 'I have frightened you- sit down and tell me all the news."']


### Learning activity: `filter()` and `map()` and `distinct()`

Lets apply some transformations onto RDDs. The following helper function will be useful to select the words from a line.

In [74]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

print(get_words("This, is a test!"))

['This', 'is', 'a', 'test']


Use `filter()` to count the number of lines which mention `war` and the number of lines which mention `peace`.

In [75]:
# How often are war and peace mentioned?
war_lines = lines.filter(lambda line: "war" in get_words(line))
print(war_lines.count())
peace_lines = lines.filter(lambda line: "peace" in get_words(line))
print(peace_lines.count())

265
104


Use `map()` to capitalise each line in the RDD, and print the first 15 capitalized lines.

In [76]:
# Capitalize each line in the RDD
lines_upper = lines.map(lambda line: line.upper())
print(lines_upper.take(15))

['                                      1869', '                                 WAR AND PEACE', '                                 BY LEO TOLSTOY', 'BK1', '                                 BOOK ONE: 1805', 'BK1|CH1', '  CHAPTER I', '', '  "WELL, PRINCE, SO GENOA AND LUCCA ARE NOW JUST FAMILY ESTATES OF THE', "BUONAPARTES. BUT I WARN YOU, IF YOU DON'T TELL ME THAT THIS MEANS WAR,", 'IF YOU STILL TRY TO DEFEND THE INFAMIES AND HORRORS PERPETRATED BY', 'THAT ANTICHRIST- I REALLY BELIEVE HE IS ANTICHRIST- I WILL HAVE', 'NOTHING MORE TO DO WITH YOU AND YOU ARE NO LONGER MY FRIEND, NO LONGER', "MY 'FAITHFUL SLAVE,' AS YOU CALL YOURSELF! BUT HOW DO YOU DO? I SEE", 'I HAVE FRIGHTENED YOU- SIT DOWN AND TELL ME ALL THE NEWS."']


Use `flatMap()` to create an RDD of the words in War and Peace and count the number of words.

In [77]:
# Split each line into words using get_words()
words = lines.flatMap(lambda line: get_words(line))
print(words.count())

573322


Finally, use `distinct()` to count the number of different words in the RDD.

In [78]:
# Count the number of distinct words
print(words.distinct().count())

19206


### Learning activity: Set like transformations

Use the functions `union()` and `intersection()` to create RDDs of lines with either war or peace mentioned, and both war and peace being mentioned. Count how many lines of each type there are and print some examples.

In [79]:
war_or_peace_lines = war_lines.union(peace_lines)
war_and_peace_lines = war_lines.intersection(peace_lines)
print(war_or_peace_lines.count())
print(war_and_peace_lines.count())

369
7


### Learning activity: `reduce()`

You have already seen three actions: `collect()` which returns all elements in the RDD, `take(n)`, which return the first `n` elements of the RDD, and `count()` which returns the number of elements in the RDD.

The action `reduce()` takes as input a function which collapses two elements into one. Use it to find the longest word in War and Peace.

In [80]:
def max(x, y):
    return x if len(x) > len(y) else y

print(words.reduce(lambda x, y: max(x, y)))

characteristically


### Bonus activity: merging filters

Find all the lines that mention both war and peace **without** using `intersection()`.

In [81]:
lines.filter(lambda line: "war" in get_words(line) and "peace" in get_words(line)).collect()

['  "To enter Russia without declaring war! I will not make peace as',
 '"but they are not even that! They are neither fit for war nor peace!',
 'peace nor war, neither an advance nor a defensive camp at the Drissa',
 'perpetual peace and the abolition of war, and secondly, by the fact',
 'war and the peace that had been concluded. "Yes, I have been much',
 'blamed," he said, "both for that war and the peace... but everything',
 "'Boyars,' I will say to them, 'I do not desire war, I desire the peace"]

### Bonus learning activity: Finding proper nouns

The Python function `str.istitle()` returns `True` if the string `str` is titlecased: the first character is uppercase and others are lowercase. Use it to:
* Find the set of distinct words in War and Peace which are titlecased
* Find the set of distinct words in War and Peace which are not titlecased

In [89]:
distinct_title_case = words.filter(lambda word: word.istitle()).distinct()
print(distinct_title_case.count())
distinct_not_title_case = words.subtract(distinct_title_case).distinct()
print(distinct_not_title_case.count())

3068
16138


The Python function `str.lower` returns a string with all characters of `str` lowercase. Use it, along with your previously generated RDD to find the set of words in War and Peace which only appear titlecased.

In [87]:
distinct_lower_case = words.filter(lambda word: word == word.lower)
print(distinct_lower_case.collect())


[]


# Key/Value pairs in Spark

### Learning activity: WordCount in Spark

Use the functions `flatMap()` and `reduceByKey()` to count the number of occurences of each word in War and Peace, and print the count of five words.

In [95]:
word_counts = lines.flatMap(get_words).map(lambda word:(word, 1)).reduceByKey(lambda a, b: a + b)
word_counts.take(5)

[('PEACE', 1), ('Leo', 1), ('BOOK', 15), ('ONE', 2), ('1805', 24)]

### Learning activity: using `groupByKey()`

Reimplement the above word count using `groupByKey()` instead of `reduceByKey()`

In [96]:
word_counts = lines.flatMap(get_words).map(lambda word:(word, 1)).groupByKey().map(lambda list : (list[0], sum(list[1])))
word_counts.take(5)

[('PEACE', 1), ('Leo', 1), ('BOOK', 15), ('ONE', 2), ('1805', 24)]

### Bonus learning activity: computing the average of each key

The pair RDD defined below `word_line_pairs` has an element for each line in War and Peace with as key the first word, and as value the line itself. Use it to compute the average length of each line for each starting word.

In [101]:
word_line_pairs = lines.filter(lambda line: len(get_words(line)) > 0).map(lambda line: (get_words(line)[0], line))     

In [104]:
test = word_line_pairs.map(lambda l : (l[0], (len(l[1]), 1))).reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])).map(lambda l; (l[0], l[1][0]/float(l[1][1])))
test.take(5)

SyntaxError: invalid syntax (<ipython-input-104-54aa3f4fe45f>, line 1)